In [ ]:
# https://www.youtube.com/watch?v=DqtlR0y0suo

In [2]:
#Sending GET request through Insomnia API to nfl/odds page. Data does not contain props. Will have to run this script 2x
import requests
import pandas as pd


get_content_url= "https://content.unabated.com/markets/game-odds/v_gameodds.json?"
get_books_url = "https://api-k.unabated.com/api/users/settings"


#Main Lines URL
Privilege_content = "https://content.unabated.com/markets/game-odds/b_gameodds.json"
#This one is when I am logged in

In [3]:
url = Privilege_content
#returns isPrivileged: true for premium sportsbooks

payload = ""
headers = {
    "authority": "content.unabated.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "dnt": "1",
    "origin": "https://unabated.com",
    "referer": "https://unabated.com/",
    "sec-ch-ua": "^\^Google",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "x-amz-cf-v-id": "dNSob7rQOiJFE_59NDSm7tuZ6ed5MA_8w8CHVqr8tGSFYpgzEUqBuQ=="
}
#We can customize the headers we include ^ I left as it came
r = requests.request("GET", url, data=payload, headers=headers)

# print(r.json())
# all good

data = r.json()

marketSources = (data["marketSources"])
teams = (data["teams"])
gameOddsEvents = (data["gameOddsEvents"])


df_sportsbooks = pd.DataFrame(marketSources)
df_teams = pd.DataFrame(teams)
df_teams=df_teams.transpose()


Transforming the SportsBooks Reference Table

In [25]:
#Removing columns for building purposes 
#Note a bunch of other sportsbooks exist but are not shown on webpage. 

# df_sb = pd.DataFrame().assign(Name=df_sportsbooks['name'],bookId=df_sportsbooks['id'],isActive=df_sportsbooks['isActive'],
# isPrivileged=df_sportsbooks['isPrivileged'],statusId=df_sportsbooks['statusId'],modifiedDate=df_sportsbooks['modifiedOn'])
df_sb = pd.DataFrame().assign(bookName=df_sportsbooks['name'],bookId=df_sportsbooks['id'],isActive=df_sportsbooks['isActive'],
statusId=df_sportsbooks['statusId'],modifiedDate=df_sportsbooks['modifiedOn'])

#Creating Binary flags 
 
df_sb['nyBook']= 0
df_sb['sharpBook']= 0


In [26]:
df_nyBooks= pd.DataFrame(
    {"bookName" : ["Caesars", "DraftKings", "FanDuel", "FanDuel - Delayed", "BetMGM", "PointsBet", "BetRivers", "WynnBet", "Resorts", "BallyBet"],
    'nyBook': [1,1,1,1,1,1,1,1,1,1]
    }
)
df_sharpBooks = pd.DataFrame(
    {"bookName" :['Unabated','Pinnacle','Pinnacle - Delayed','Bookmaker','Prophet Exchange'],
    'sharpBook': [1,1,1,1,1]
    }
)

In [27]:
df_sb=df_sb.set_index('bookName')
df2 = df_nyBooks.set_index('bookName')
df_sb.update(df2)

df3=df_sharpBooks.set_index('bookName')
df_sb.update(df3)

In [28]:
df_sb

#This is the Sportsbook Reference table. Might also want to change the number format for the dates

,bookId,isActive,statusId,modifiedDate,nyBook,sharpBook
bookName,,,,,,
Circa - Delayed,68,True,2,2022-12-21T19:37:56.422057,0.0,0.0
WynnBet,33,True,2,2022-10-31T16:22:07.064598,1.0,0.0
Sugarhouse,27,True,3,2022-12-18T20:14:17.691736,0.0,0.0
Sporttrade,67,True,1,2022-12-17T22:15:38.503501,0.0,0.0
Sports Interaction,69,True,2,2022-12-21T19:37:56.422057,0.0,0.0
...,...,...,...,...,...,...
BallyBet,38,True,3,2021-07-07T02:13:52.730139,1.0,0.0
Barstool,36,True,3,2021-07-07T02:13:52.730139,0.0,0.0
Bet365,14,True,3,2021-07-07T02:13:52.730139,0.0,0.0


In [29]:
#Limit my calculations to nybooks only

nybooks_df = df_sb.loc[df_sb['nyBook'] == 1]
nySharpbooks_df = df_sb.loc[(df_sb['nyBook'] == 1) | (df_sb['sharpBook'] == 1)]

In [34]:
# move the bookName Index to be a column and replace with a continuous interger iD

# Step 1: Reset the index
nySharpbooks_df = nySharpbooks_df.reset_index()

# Step 2: Rename the 'index' column to 'ID'
nySharpbooks_df = nySharpbooks_df.rename(columns={'index': 'ID'})

# Step 3: Assign a new continuous integer index
nySharpbooks_df = nySharpbooks_df.reset_index(drop=True)

In [36]:
nySharpbooks_df

,bookName,bookId,isActive,statusId,modifiedDate,nyBook,sharpBook
0,WynnBet,33,True,2,2022-10-31T16:22:07.064598,1.0,0.0
1,Unabated,49,True,0,2022-03-16T21:38:37.525262,0.0,1.0
2,Pinnacle - Delayed,58,True,2,2022-08-17T12:18:20.442277,0.0,1.0
3,FanDuel - Delayed,57,True,2,2022-06-09T14:45:56.826297,1.0,0.0
4,DraftKings,1,True,1,2022-06-28T15:42:41.593641,1.0,0.0
5,BetMGM,4,True,3,2022-07-28T13:36:35.331082,1.0,0.0
6,PointsBet,3,True,2,2022-08-27T03:32:48.307845,1.0,0.0
7,Prophet Exchange,66,True,1,2022-11-16T22:12:44.347255,0.0,1.0
8,Pinnacle,7,True,2,2022-10-24T15:46:35.636137,0.0,1.0
9,FanDuel,2,True,1,2022-06-28T16:18:24.527118,1.0,0.0


In [ ]:
df_sb.to_csv('sportsbooksTable.csv')

________________________________________________________________________________________


Now we create the Teams table. 

The eventId is the unique gameID which two teams will share. Not positive on what sideId is yet, but it distinguishes one team from another inside of a cell. 


In [9]:
#slimming down

df_teams_refined = pd.DataFrame().assign(teamId=df_teams['id'],teamName=df_teams['name'],abbreviation=df_teams['abbreviation'],
eventId=df_teams['eventId'],sideId=df_teams['sideId'],leagueId=df_teams['leagueId'],divisionId=df_teams['divisionId'],
modifiedDate=df_teams['modifiedOn'])

df_teams_refined
#This is our Teams Table

,teamId,teamName,abbreviation,eventId,sideId,leagueId,divisionId,modifiedDate
2,2,Atlanta Falcons,ATL,21959,1,1,8,0001-01-01T00:00:00
30,30,Tampa Bay Buccaneers,TB,21959,0,1,8,0001-01-01T00:00:00
19,19,New England Patriots,NE,21960,0,1,1,0001-01-01T00:00:00
4,4,Buffalo Bills,BUF,21960,1,1,1,0001-01-01T00:00:00
6,6,Chicago Bears,CHI,21961,1,1,7,0001-01-01T00:00:00
...,...,...,...,...,...,...,...,...
535,535,NC Asheville,UNCA,39048,1,4,None,0001-01-01T00:00:00
1073,1073,South Dakota State,SDS,40030,1,2,10,0001-01-01T00:00:00
1175,1175,North Dakota State,NDS,40030,0,2,10,0001-01-01T00:00:00
689,689,TCU,TCU,40492,0,2,9,0001-01-01T00:00:00


In [33]:
df_teams_refined.to_csv('df_teams_ref_out.csv')



__________________________________________________________________________________

Now we tackle the GameOdd Table

In [41]:
data = []
for i in gameOddsEvents:
  for j in gameOddsEvents[i]:
    event_teams = j['eventTeams']
    id_0 = event_teams['0']['id']
    id_1 = event_teams['1']['id']
    
    # Extract the americanPrice values from gameOddsMarketSourcesLines
    game_odds_market_sources_lines = j['gameOddsMarketSourcesLines']
    for market_source, market_data in game_odds_market_sources_lines.items():
      siX= market_source
      for bet_type, bet_data in market_data.items():
            american_price = bet_data['americanPrice']
            marketLineId= bet_data['marketLineId']
            marketId= bet_data['marketId']
            marketSourceId= bet_data['marketSourceId']
            points= bet_data['points']
            price= bet_data['price']
            sourcePrice= bet_data['sourcePrice']
            btX = bet_type
            # Store the extracted data in a tuple
            row = (i, j['eventId'],j['eventStart'],j['eventEnd'] ,id_0, id_1, siX, btX, marketLineId, marketSourceId,
            marketId, points,american_price, price, sourcePrice)
            data.append(row)



# Create the DataFrame with the extracted data
df_MainLineOdds = pd.DataFrame(data, columns=['lgX', 'eventId','eventStart','eventEnd','Teamid_0', 'Teamid_1', 'siX', 'btX', 'marketLineId','marketSourceId','marketId','points','american_price','price','sourcePrice'])
df_MainLineOdds['lgX_league']= df_MainLineOdds['lgX'].str.get(2)
df_MainLineOdds['lgX_pt']= df_MainLineOdds['lgX'].str.get(6)
df_MainLineOdds['lgX_stage']= df_MainLineOdds['lgX'].str[8:]
df_MainLineOdds['siX_side']= df_MainLineOdds['siX'].str.get(2)

df_MainLineOdds

,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,marketLineId,marketSourceId,marketId,points,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side
0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,67201420,1,474203,-4.0,-110,-110,1.909091,1,1,pregame,1
1,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,249616595,1,1578122,40.5,-110,-110,1.909091,1,1,pregame,1
2,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,67201450,1,667433,0.0,-195,-195,1.512821,1,1,pregame,1
3,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,270812471,2,474203,-4.5,-104,-104,-104.000000,1,1,pregame,1
4,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt3,270812481,2,1578122,40.5,-110,-110,-110.000000,1,1,pregame,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56360,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms17:an0,bt2,279269637,17,1645395,-2.5,-125,-125,1.800000,1,3,live,0
56361,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms27:an0,bt3,279271964,27,1645397,24.5,-115,-115,1.869570,1,3,live,0
56362,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms27:an0,bt2,279271952,27,1645395,-2.5,-124,-124,1.806450,1,3,live,0
56363,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms21:an0,bt1,279270163,21,1645399,NaN,-200,-200,-200.000000,1,3,live,0


In [42]:
bet_type = {
    'bt1': 'Money_Line',
    'bt2':'Spread',
    'bt3': 'Total_Combined'}

#There is a bt4 : 'Combined' I have not seen it exisst in the data

In [32]:
df_MainLineOdds.to_csv('MainLineOdds.csv')

Write to MainLinesMaster

In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\ammoo\OneDrive\Desktop\PythonOutData\Main Lines Master.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_MainLineOdds.to_excel(writer, sheet_name='MainLineOdds')
df_teams_refined.to_excel(writer, sheet_name='Teams')
df_sb.to_excel(writer, sheet_name='Books')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

c:\Users\ammoo\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Create Table where only bets for nySharpbooks_df

In [143]:
filtered_df = df_MainLineOdds[df_MainLineOdds['marketSourceId'].isin(nySharpbooks_df['bookId'])].copy()

In [68]:
filtered_df

,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,marketLineId,marketSourceId,marketId,points,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side
0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,67201420,1,474203,-4.0,-110,-110,1.909091,1,1,pregame,1
1,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,249616595,1,1578122,40.5,-110,-110,1.909091,1,1,pregame,1
2,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,67201450,1,667433,0.0,-195,-195,1.512821,1,1,pregame,1
3,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,270812471,2,474203,-4.5,-104,-104,-104.000000,1,1,pregame,1
4,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt3,270812481,2,1578122,40.5,-110,-110,-110.000000,1,1,pregame,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56356,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms1:an0,bt3,279269731,1,1645397,24.5,-120,-120,1.833333,1,3,live,0
56357,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms1:an0,bt2,279269664,1,1645395,-2.5,-120,-120,1.833333,1,3,live,0
56358,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms1:an0,bt1,279269752,1,1645399,0.0,-175,-175,1.571429,1,3,live,0
56359,lg1:pt3:live,40525,2023-01-07T21:30:00,None,16,23,si0:ms17:an0,bt3,279269724,17,1645397,25.5,-117,-117,1.854700,1,3,live,0


Insert the Bet Type from the bet type dictionary and TeamName_1, sideId_team1, TeamName_0, sideId_team0 from Teams table. We identify which team the bet is placed for by comparing the bet's siX_side to the appropriate sideId_team0/1 

In [144]:
# Insert the bet_type

# filtered_df['bet_type'] = filtered_df['btX'].map(bet_type)
filtered_df.loc[:, 'bet_type'] = filtered_df['btX'].map(bet_type)

In [145]:
#Insert the TeamNames and side(home/away) for Teamid_0 and Teamid_1
# Merge the two dataframes on teamId
filtered_df = df_teams_refined.merge(filtered_df, left_on='teamId', right_on='Teamid_0')

# Rename the columns so they don't overwrite each other
filtered_df = filtered_df.rename(columns={'teamName':'TeamName_0', 'abbreviation':'TeamAbbrev_0','sideId':'sideId_team0'})

# Drop the now unnecessary merged columns column
filtered_df = filtered_df.drop(columns={'teamId','TeamAbbrev_0','eventId_x','leagueId','divisionId','modifiedDate'})


#repeat for Teamid_1
filtered_df = filtered_df.merge(df_teams_refined, left_on='Teamid_1', right_on='teamId')
filtered_df = filtered_df.rename(columns={'teamName':'TeamName_1', 'abbreviation':'TeamAbbrev_1','sideId':'sideId_team1'})
filtered_df = filtered_df.drop(columns=['teamId','TeamAbbrev_1','leagueId','divisionId','modifiedDate','eventId'])
filtered_df = filtered_df.rename(columns={'eventId_y' : 'eventId'})

In [71]:
filtered_df

,TeamName_0,sideId_team0,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,...,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side,bet_type,TeamName_1,sideId_team1
0,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,...,-110,-110,1.909091,1,1,pregame,1,Spread,Atlanta Falcons,1
1,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,...,-110,-110,1.909091,1,1,pregame,1,Total_Combined,Atlanta Falcons,1
2,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,...,-195,-195,1.512821,1,1,pregame,1,Money_Line,Atlanta Falcons,1
3,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,...,-104,-104,-104.000000,1,1,pregame,1,Spread,Atlanta Falcons,1
4,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt3,...,-110,-110,-110.000000,1,1,pregame,1,Total_Combined,Atlanta Falcons,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20924,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,...,-110,-110,1.909091,2,3,pregame,0,Total_Combined,Georgia,1
20925,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt1,...,240,240,3.400000,2,3,pregame,0,Money_Line,Georgia,1
20926,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt2,...,-105,-105,1.952381,2,3,pregame,1,Spread,Georgia,1
20927,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,...,-285,-285,1.350877,2,3,pregame,1,Money_Line,Georgia,1


In [146]:
filtered_df.columns

Index(['TeamName_0', 'sideId_team0', 'lgX', 'eventId', 'eventStart',
       'eventEnd', 'Teamid_0', 'Teamid_1', 'siX', 'btX', 'marketLineId',
       'marketSourceId', 'marketId', 'points', 'american_price', 'price',
       'sourcePrice', 'lgX_league', 'lgX_pt', 'lgX_stage', 'siX_side',
       'bet_type', 'TeamName_1', 'sideId_team1'],
      dtype='object')

Calc ImpliedProb%

In [61]:
# I am going to create a column called Implied Probability. To calculate implied probability, 
# we are going to look to the american_price column. 

# These are the rules: 
# btX = bt1
# If the odds are positive, do 100 / (odds + 100)
# So for +400, it would be 100 / (400 + 100) = 100/500 = 20% Implied WP
# If the odds are negative, do odds / (odds + 100)
# So for -300, it would be 300 / (300 + 100) = 300/400 = 75% Implied WP -->


# def calculate_implied_probability(row):
#     if row['american_price'] > 0:
#         return 100 / (row['american_price'] + 100)
#     else:
#         return row['american_price'] / (row['american_price'] + 100)

# filtered_df['ImpliedProb%'] = filtered_df.apply(calculate_implied_probability, axis=1)



#We assume that implied probability is calculated the same for all bet types

# def calc_implied_probability_MoneyLine(row):
#     if row['btX'] == 'bt1':
#         if row['american_price'] > 0:
#             return 100 / (row['american_price'] + 100)
#         elif row['american_price'] < 0:
#             return row['american_price'] / (row['american_price'] + 100)
#     return None

# filtered_df['ImpliedProb%'] = filtered_df.apply(calc_implied_probability_MoneyLine, axis=1)


In [147]:
def implied_probability(row):

        if row['american_price'] > 0:
            return 1 / (row['american_price']/100 + 1) *100
        elif row['american_price'] < 0:
            return 1/(1+100/abs(row['american_price'])) *100
    

filtered_df['ImpliedProb%'] = filtered_df.apply(implied_probability, axis=1)

In [62]:
filtered_df
#This contains every bet for ny and sharpbooks. Next i should add book names

,TeamName_0,sideId_team0,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,...,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side,bet_type,TeamName_1,sideId_team1,ImpliedProb%
0,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt2,...,-110,1.909091,1,1,pregame,1,Spread,Atlanta Falcons,1,52.380952
1,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt3,...,-110,1.909091,1,1,pregame,1,Total_Combined,Atlanta Falcons,1,52.380952
2,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms1:an0,bt1,...,-195,1.512821,1,1,pregame,1,Money_Line,Atlanta Falcons,1,66.101695
3,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt2,...,-104,-104.000000,1,1,pregame,1,Spread,Atlanta Falcons,1,50.980392
4,Tampa Bay Buccaneers,0,lg1:pt1:pregame,21959,2023-01-08T18:00:00,None,30,2,si1:ms2:an0,bt3,...,-110,-110.000000,1,1,pregame,1,Total_Combined,Atlanta Falcons,1,52.380952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20924,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt3,...,-110,1.909091,2,3,pregame,0,Total_Combined,Georgia,1,52.380952
20925,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si0:ms1:an0,bt1,...,240,3.400000,2,3,pregame,0,Money_Line,Georgia,1,29.411765
20926,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt2,...,-105,1.952381,2,3,pregame,1,Spread,Georgia,1,51.219512
20927,TCU,0,lg2:pt3:pregame,40492,2023-01-10T00:30:00,None,689,625,si1:ms1:an0,bt1,...,-285,1.350877,2,3,pregame,1,Money_Line,Georgia,1,74.025974


Merge bookName into df_filtered

In [148]:
filtered_df = filtered_df.merge(nySharpbooks_df, left_on='marketSourceId', right_on='bookId')


In [129]:
filtered_df

,marketLineId,eventId,eventStart,eventEnd,Teamid_0,TeamName_0,sideId_team0,Teamid_1,TeamName_1,sideId_team1,...,american_price,price,sourcePrice,ImpliedProb%,marketSourceId,bookName,bookId,bookIsActive,nyBook,sharpBook
0,67201420,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
1,249616595,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
2,67201450,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-195,-195,1.512821,66.101695,1,DraftKings,1,True,1.0,0.0
3,67201421,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
4,249616613,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20924,272854935,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,400,400,400.000000,20.000000,66,Prophet Exchange,66,True,0.0,1.0
20925,272854943,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-102,-102,-102.000000,50.495050,66,Prophet Exchange,66,True,0.0,1.0
20926,272854937,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-101,-101,-101.000000,50.248756,66,Prophet Exchange,66,True,0.0,1.0
20927,272854921,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-430,-430,-430.000000,81.132075,66,Prophet Exchange,66,True,0.0,1.0


In [149]:
# Drop & rename columns. 

filtered_df= filtered_df.drop(columns=['modifiedDate', 'statusId','siX'])

# Rename Columns 

filtered_df= filtered_df.rename(columns={
    'isActive':'bookIsActive',
    'siX_side': 'bet_side'})



In [150]:
#Insert betPeriod column

ptX_definiton = {
    '1':'FullGame',
    '2':'Half_1',
    '3':'Half_2',
    '4':'Quarter_1',
    '5':'Quarter_2',
    '6':'Quarter_3',
    '7':'Quarter_4'}


filtered_df.loc[:, 'betPeriod'] = filtered_df['lgX_pt'].map(ptX_definiton)

In [151]:


#Reorder to make it easier to read

easiest_to_read_column_order = ['marketLineId','eventId', 'eventStart', 'eventEnd', 'Teamid_0', 'TeamName_0', 'sideId_team0', 
'Teamid_1', 'TeamName_1','sideId_team1', 'bet_side','lgX', 'lgX_league', 'lgX_pt','betPeriod', 'lgX_stage',   
'marketId', 'btX', 'bet_type', 'points', 'american_price', 
'price', 'sourcePrice', 'ImpliedProb%','marketSourceId', 'bookName', 'bookId', 'bookIsActive', 'nyBook', 'sharpBook']

filtered_df= filtered_df.reindex(columns=easiest_to_read_column_order)

In [152]:
filtered_df

,marketLineId,eventId,eventStart,eventEnd,Teamid_0,TeamName_0,sideId_team0,Teamid_1,TeamName_1,sideId_team1,...,american_price,price,sourcePrice,ImpliedProb%,marketSourceId,bookName,bookId,bookIsActive,nyBook,sharpBook
0,67201420,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
1,249616595,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
2,67201450,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-195,-195,1.512821,66.101695,1,DraftKings,1,True,1.0,0.0
3,67201421,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
4,249616613,21959,2023-01-08T18:00:00,None,30,Tampa Bay Buccaneers,0,2,Atlanta Falcons,1,...,-110,-110,1.909091,52.380952,1,DraftKings,1,True,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20924,272854935,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,400,400,400.000000,20.000000,66,Prophet Exchange,66,True,0.0,1.0
20925,272854943,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-102,-102,-102.000000,50.495050,66,Prophet Exchange,66,True,0.0,1.0
20926,272854937,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-101,-101,-101.000000,50.248756,66,Prophet Exchange,66,True,0.0,1.0
20927,272854921,40492,2023-01-10T00:30:00,None,689,TCU,0,625,Georgia,1,...,-430,-430,-430.000000,81.132075,66,Prophet Exchange,66,True,0.0,1.0


Write to MainLines_Master.v2


In [135]:
filtered_df.to_csv('MainLineOdds.v2.csv')

Calculate EV for each record (marketLineID) that shares a marketId against a benchmark 
 



1.  group items by TeamName_0, bet_type, betPeriod... this will return over/under bets for each book needed in No vig calc
2. Calculate the benchmark No Vig 
Here’s the formula:




Packers: 45.45% / (45.45% + 58.33%) = 44%

Vikings: 58.33% / (58.33% + 45.45%) = 56%

That gets you to 100%.




Ev of Bet = 
Benchmark No Vig Win% * Profit if win / (100-Amountt Staked *(100-Benchmark No Vig Win %)/100)

In [ ]:


# # For records with nyBook =1:
# if bet_side = 0
# TeamName_0 Implied Probability / (TeamName_0 IP + TeamName_1 IP)
# else
# TeamName_1 Implied Probability / (TeamName_1 IP + TeamName_0 IP)





#This is not what i need. I need to pull odds data from 2 records with the same attributes which identify that pair from the other
# group by bookName

In [138]:
def calculate_probability(df):
  # For records with nyBook = 1
  df = df[df['nyBook'] == 1]
  
  # If bet_side = 0
  if df['bet_side'].iloc[0] == 0:
    return df['TeamName_0']['ImpliedProb%'] / (df['TeamName_0']['ImpliedProb%'] + df['TeamName_1']['ImpliedProb%'])
  else:
    return df['TeamName_1']['ImpliedProb%'] / (df['TeamName_1']['ImpliedProb%'] + df['TeamName_0']['ImpliedProb%'])

# Group the records by TeamName_0, bet_type, and betPeriod
grouped_df = df.groupby(['TeamName_0', 'bet_type', 'betPeriod'])

# Apply the function to each group
result = grouped_df.apply(calculate_probability)

NameError: name 'df' is not defined

In [137]:
grouped_df = filtered_df.groupby(['TeamName_0', 'bet_type', 'betPeriod',])

def calc_prob(df):
    if df['nyBook'] == 1:
        if df['bet_side'] == 0:
            return df['ImpliedProb%'] / (df['ImpliedProb%'] + df['sideId_team1'])
        else:
            return df['sideId_team1'] / (df['sideId_team1'] + df['ImpliedProb%'])
    else:
        # do something else if nyBook is not 1
        return df['ImpliedProb%']

result_df = grouped_df.apply(calc_prob)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [164]:
filtered_df.columns

Index(['marketLineId', 'eventId', 'eventStart', 'eventEnd', 'Teamid_0',
       'TeamName_0', 'sideId_team0', 'Teamid_1', 'TeamName_1', 'sideId_team1',
       'bet_side', 'lgX', 'lgX_league', 'lgX_pt', 'betPeriod', 'lgX_stage',
       'marketId', 'btX', 'bet_type', 'points', 'american_price', 'price',
       'sourcePrice', 'ImpliedProb%', 'marketSourceId', 'bookName', 'bookId',
       'bookIsActive', 'nyBook', 'sharpBook'],
      dtype='object')

In [155]:
def calculate_probability(filtered_df):
    if filtered_df['nyBook'].iloc[0] != 1:
        return
    if filtered_df['bet_type'].iloc[0] == 0:
        return filtered_df['ImpliedProb%'].iloc[0] / (filtered_df['ImpliedProb%'].iloc[0] + filtered_df['ImpliedProb%'].iloc[1])
    else:
        return filtered_df['ImpliedProb%'].iloc[1] / (filtered_df['ImpliedProb%'].iloc[1] + filtered_df['ImpliedProb%'].iloc[0])

filtered_df2 = filtered_df.groupby(['TeamName_0', 'bet_type', 'betPeriod', 'bookName']).apply(calculate_probability)
filtered_df = filtered_df.apply(calculate_probability)

KeyError: 'nyBook'

I want to bin every record into groups where they share the same TeamName_0, bet_type, betPeriod, bookName. I want to apply the function below to each bin and store the result to be stored in a column called 'No_Vig'

def calculate_probability(filtered_df):
    if filtered_df['bet_type'].iloc[0] == 0:
        return filtered_df['ImpliedProb%'].iloc[0] / (filtered_df['ImpliedProb%'].iloc[0] + filtered_df['ImpliedProb%'].iloc[1])
    else:
        return filtered_df['ImpliedProb%'].iloc[1] / (filtered_df['ImpliedProb%'].iloc[1] + filtered_df['ImpliedProb%'].iloc[0])

In [166]:
def calculate_NoVig(filtered_df):
    if filtered_df['bet_side'].iloc[0] == 0:
        return filtered_df['ImpliedProb%'].iloc[0] / (filtered_df['ImpliedProb%'].iloc[0] + filtered_df['ImpliedProb%'].iloc[1])
    else:
        return filtered_df['ImpliedProb%'].iloc[1] / (filtered_df['ImpliedProb%'].iloc[1] + filtered_df['ImpliedProb%'].iloc[0])

filtered_df['No_Vig'] = filtered_df.groupby(
    ['TeamName_0', 'bet_type', 'betPeriod', 'bookName'])['ImpliedProb%'].apply(calculate_NoVig)

KeyError: 'bet_side'

In [167]:
def calculate_NoVig(filtered_df):
    if filtered_df['bet_side'].iloc[0] == 0:
        return filtered_df['ImpliedProb%'].iloc[0] / (filtered_df['ImpliedProb%'].iloc[0] + filtered_df['ImpliedProb%'].iloc[1])
    else:
        return filtered_df['ImpliedProb%'].iloc[1] / (filtered_df['ImpliedProb%'].iloc[1] + filtered_df['ImpliedProb%'].iloc[0])

In [168]:
filtered_df = filtered_df.assign(
    No_Vig=filtered_df.groupby(['TeamName_0', 'bet_type', 'betPeriod', 'bookName']).apply(calculate_NoVig))

TypeError: incompatible index of inserted column with frame index

In [169]:
def calculate_probability(row):
    if row['bet_side'] == 0:
        return row['ImpliedProb%'] / (row['ImpliedProb%'] + row['ImpliedProb%'].shift())
    else:
        return row['ImpliedProb%'].shift() / (row['ImpliedProb%'].shift() + row['ImpliedProb%'])

filtered_df['No_Vig'] = filtered_df.apply(calculate_probability, axis=1)

AttributeError: 'float' object has no attribute 'shift'

In [179]:
filtered_df['ImpliedProb%'].info()

<class 'pandas.core.series.Series'>
Int64Index: 20929 entries, 0 to 20928
Series name: ImpliedProb%
Non-Null Count  Dtype  
--------------  -----  
20927 non-null  float64
dtypes: float64(1)
memory usage: 327.0 KB


In [170]:
def calculate_NoVig2(row):
    if row['bet_side'].iloc[0] == 0:
        return row['ImpliedProb%'].iloc[0] / (row['ImpliedProb%'].iloc[0] + row['ImpliedProb%'].iloc[1])
    else:
        return row['ImpliedProb%'].iloc[1] / (row['ImpliedProb%'].iloc[1] + row['ImpliedProb%'].iloc[0])



In [176]:
filtered_df['No_Vig'] = 0


In [174]:
filtered_df.groupby([
    'TeamName_0', 'bet_type', 'betPeriod', 'bookName'])['No_Vig'].apply(lambda x: x.apply(calculate_NoVig2))

TypeError: 'int' object is not subscriptable

In [177]:
for name, group in filtered_df.groupby(['TeamName_0', 'bet_type', 'betPeriod', 'bookName']):
    group['No_Vig'] = group.apply(calculate_NoVig2, axis=1)

AttributeError: 'str' object has no attribute 'iloc'

In [180]:
# Initialize a list to store the 'No_Vig' values
no_vig_values = []

# Loop through the sub-group and calculate the 'No_Vig' value for each row
for index, row in group.iterrows():
    if row['bet_side'] == 0:
        no_vig_value = row['ImpliedProb%'] / (row['ImpliedProb%'] + row['ImpliedProb%'].shift(1))
    else:
        no_vig_value = row['ImpliedProb%'] / (row['ImpliedProb%'] + row['ImpliedProb%'].shift(-1))
        
    no_vig_values.append(no_vig_value)

return no_vig_values

AttributeError: 'float' object has no attribute 'shift'

In [186]:
def calculate_NoVig(row):
    if row['bet_side'] == 0:
        # find the record with the other team and get its ImpliedProb%
        other_team_ip = filtered_df.loc[(filtered_df['TeamName_0'] == row['TeamName_1']) & 
        (filtered_df['TeamName_1'] == row['TeamName_0']) & (filtered_df['bet_side'] == 1)]['ImpliedProb%'].values[0]
        return row['ImpliedProb%'] / (row['ImpliedProb%'] + other_team_ip)
    else:
        other_team_ip = filtered_df.loc[(filtered_df['TeamName_0'] == row['TeamName_1']) & 
        (filtered_df['TeamName_1'] == row['TeamName_0']) & (filtered_df['bet_side'] == 0)]['ImpliedProb%'].values[0]
        return row['ImpliedProb%'] / (row['ImpliedProb%'] + other_team_ip)

In [187]:
filtered_df['No_Vig'] = filtered_df.apply(calculate_NoVig, axis=1)

IndexError: index 0 is out of bounds for axis 0 with size 0